<a href="https://colab.research.google.com/github/Trim0500/COMP433-Neural-Networks-Project/blob/Task%2FPCOMP433-BERT-MUltilingual-Work-And_Submission-Template-File/%5BCOMP433%5D_BERT_Submission_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q keras-nlp --upgrade

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import keras_nlp
import keras
import tensorflow as tf
import gc

keras.mixed_precision.set_global_policy("mixed_float16")

In [4]:
df_train = pd.read_csv("train.csv")
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [5]:
def split_labels(x, y):
    return (x[0], x[1]), y

- Use this block if the preprocessor used for the model is changed from bert base multi

In [9]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_multi", sequence_length=512)

def get_max_token_length(dataset, preprocessor):
  token_lengths = []

  for hypothesis, premise in dataset[["hypothesis", "premise"]].values:

    encoded = preprocessor([hypothesis, premise])
    encoded_np = encoded["padding_mask" ]. numpy()
    non_padding_count = encoded_np.sum()
    token_lengths.append(non_padding_count)

  return max(token_lengths)

max_token_length = get_max_token_length(df_train, preprocessor)

print(f"Maximum token length in the dataset: {max_token_length}")

100%|██████████| 511/511 [00:00<00:00, 711kB/s]


100%|██████████| 548/548 [00:00<00:00, 774kB/s]


100%|██████████| 972k/972k [00:00<00:00, 4.83MB/s]


Maximum token length in the dataset: 260


Use this block if you want to train a new model instance

In [11]:
batch_size = 28
buffer_size = batch_size * 10

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_train[["hypothesis", "premise"]].values,
            df_train["label"].values
        )
    )
)

print(f"Size of the train dataset: {len(training_dataset)}")

train_preprocessed = training_dataset.shuffle(buffer_size=buffer_size).map(split_labels, tf.data.AUTOTUNE).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

Size of the train dataset: 12120


In [17]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_multi", sequence_length=265)

bert_train_set = (train_preprocessed.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE))

classifier = keras_nlp.models.BertClassifier.from_preset("bert_base_multi", preprocessor=None, num_classes=3)
classifier.compile(optimizer=keras.optimizers.Adam(4e-05), metrics=['accuracy'])
history = classifier.fit(bert_train_set, epochs=5)

Epoch 1/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 108s 73ms/step - accuracy: 0.4990 - loss: 0.9844
Epoch 2/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 31s 73ms/step - accuracy: 0.6699 - loss: 0.7514
Epoch 3/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 31s 73ms/step - accuracy: 0.7667 - loss: 0.5675
Epoch 4/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 31s 73ms/step - accuracy: 0.8406 - loss: 0.4135
Epoch 5/5
432/432 ━━━━━━━━━━━━━━━━━━━━ 31s 73ms/step - accuracy: 0.8838 - loss: 0.3056


- Run this block if you wish to save the model trained on the above block (name the file according to the hyperparameters used)

In [18]:
classifier.save_weights("Bert_multi_lr43-05_B28.weights.h5")

Use this block if you have a prexisting saved weights file that you want to use

In [6]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_base_multi", preprocessor=None, num_classes=3)
classifier.load_weights("Bert_multi_lr43-05_B28.weights.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 408 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 404 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


The block above only reutilizes the weights for prediction, thus the mismatch optimizer warnings can be ignored as no further additional training is utilized for this notebook

### Precitions on test data for submission

In [7]:
df_test = pd.read_csv("test.csv")
df_test.head()
df_test["label"] = [0] * len(df_test)

In [8]:
def split_labels(x, y):
    return (x[0], x[1]), y

testing_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_test[["hypothesis", "premise"]].values,
            df_test["label"].values
        )
    )
)

print(len(testing_dataset))
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_multi", sequence_length=265)

test_preprocessed = testing_dataset.map(split_labels, tf.data.AUTOTUNE).batch(len(testing_dataset), drop_remainder=False).cache().prefetch(tf.data.AUTOTUNE)
bert_test_set = (test_preprocessed.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE))




5195


In [9]:
predictions = classifier.predict(bert_test_set)

1/1 ━━━━━━━━━━━━━━━━━━━━ 35s 35s/step


In [10]:
predicted_classes = tf.argmax(predictions, axis=1)
predicted_classes_np = predicted_classes.numpy()
print(predicted_classes_np)

[2 1 0 ... 1 0 2]


In [11]:
submission = df_test.id.copy().to_frame()
submission["prediction"] = predicted_classes_np
submission.to_csv("submission.csv", index=False)

submission

,id,prediction
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,2
...,...,...
5190,5f90dd59b0,1
5191,f357a04e86,1
5192,1f0ea92118,1
5193,0407b48afb,0
